## Integrin
## Slow modes using DMap

## setup

In [4]:
import os
import sys
import math
import time
import pickle
import copy
import mdtraj
import MDAnalysis as mda
#import pyemma
#import pyemma.util.contexts
import networkx as nx
import random
#import parmed as pmd
import acpype
import pandas as pd
from MDAnalysis.analysis import distances


import numpy as np
from scipy.linalg import eigh
from scipy.spatial.distance import pdist
from scipy.interpolate import interp1d
import matplotlib as mpl
from matplotlib import ticker
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#import torch
#import torch.optim as optim
#import torch.nn as nn
#from torchviz import make_dot
#from torch.utils.data import Dataset, TensorDataset, DataLoader
#from torch.utils.data.dataset import random_split
from tqdm import tqdm

#import deeptime
#from deeptime.decomposition import TICA
#from deeptime.clustering import KMeans
#import mdshare

from sklearn.preprocessing import MinMaxScaler

from jax import numpy as jnp, jit, vmap
from MDAnalysis.analysis import rms #diffusionmap, align, rms

import nglview as nv
import io
from PIL import Image 

no display found. Using non-interactive Agg backend


In [5]:
#from env_snrv.snrv.snrv.snrv import Snrv, load_snrv
#from codes.dMap_JAX import dMap_jax
#from codes.dMap_JAX import diff_plots

#### plot settings

In [6]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
font = {'weight' : 'normal',
        'size'   : 25}

plt.rc('font', **font)

#### load all-atom integrin trajectories (different integrin type)

In [ ]:
universe_aa_trajs = []

for rep in ['bent', 'int1', 'int2', 'open']: #, '0d01_10',  '0d01_5']:

    file_aa_traj_bent = 'integrin/all-atom/' + rep + '_0_500_ns.pdb'

    universe_aa_trajs.append( mda.Universe(file_aa_traj_bent) )
    


In [ ]:
universe_aa_trajs[0].atoms.n_atoms

In [ ]:
np.arange(1008)

#### visualize all-atom trajectories

In [ ]:
view = nv.show_mdanalysis(universe_aa_trajs[3])

# clear representations
#view.clear_representations()

#view.add_representation('spacefill')
view._remote_call("setSize", target="Widget", args=["1000px", "500px"])
#?view.center_view()

# specify color
view.add_representation('spacefill', selection=np.arange(1008), color='blue', radius=1.5, opacity=0.85)
#view.add_representation('spacefill', selection=np.arange(1008,1770), color='red', radius=1.5, opacity=0.85)
view.add_representation('cartoon', color='black')

# specify color
#view.add_representation('spacefill', selection=list(np.arange(169)), color='blue', radius=5, opacity=0.85)
#view.add_representation('spacefill', selection=list(np.arange(169, 300)), color='red', radius=5)

#view.add_representation('spacefill', selection="index 100 to 299", color='red')

#view.component_1.add_surface(opacity=0.3)
#view.download_image()
#view.render_image(factor=6) # higher is better
# need to run below command in different notebook cell
#view._display_image()

view

In [8]:
range(300)

range(0, 300)

#### write CG from all atom

In [ ]:
### create a universe for CG model [from example in MDAnalysis]

n_residues = 300
n_atoms = n_residues

# create resindex list
resindices = np.arange(n_residues)

# all water molecules belong to 1 segment
segindices = [0] * n_residues

# create the Universe
cg_universe = mda.Universe.empty(n_atoms,
                                 n_residues=n_residues,
                                 atom_resindex=resindices,
                                 residue_segindex=segindices,
                                 trajectory=True) # necessary for adding coordinates

In [70]:
#cg_universe.trajectory[0].positions

In [85]:
### topology attributes
from MDAnalysis.coordinates.memory import MemoryReader

cg_universe.add_TopologyAttr('name', ['CG']*n_residues)
cg_universe.add_TopologyAttr('type', ['CG']*n_residues)
cg_universe.add_TopologyAttr('resid', list(range(1, n_residues+1)))
cg_universe.add_TopologyAttr('segid', ['INT'])

#### frame 0 positions

####
select_map = 'ba_open'
universe_index = 3

aa_cg_map = pd.read_csv('integrin/all-atom/' + select_map + '.dat', names=['map'])

k = 0
cg_traj_pos = []

cg_pos_frame_0 = []
for i in range(1, 301):
    
    idx = np.where(aa_cg_map.map == i)
        
    cg_pos_frame_0.append( universe_aa_trajs[universe_index].trajectory[0].positions[idx].mean(axis=0) )
        

cg_pos_frame_0 = np.array(cg_pos_frame_0)

cg_universe.atoms.positions = cg_pos_frame_0

##### visualize

In [ ]:
view = nv.add_mdanalysis(cg_universe)

# clear representations
#view.clear_representations()

#view.add_representation('spacefill')
view._remote_call("setSize", target="Widget", args=["1000px", "500px"])
#?view.center_view()

# specify color
view.add_representation('spacefill', selection=np.arange(169), color='blue', radius=1.5, opacity=0.85)
view.add_representation('spacefill', selection=np.arange(169,300), color='red', radius=1.5, opacity=0.85)
view.add_representation('cartoon', color='black')

# specify color
#view.add_representation('spacefill', selection=list(np.arange(169)), color='blue', radius=5, opacity=0.85)
#view.add_representation('spacefill', selection=list(np.arange(169, 300)), color='red', radius=5)

#view.add_representation('spacefill', selection="index 100 to 299", color='red')

#view.component_1.add_surface(opacity=0.3)
#view.download_image()
#view.render_image(factor=6) # higher is better
# need to run below command in different notebook cell
#view._display_image()

view

In [87]:

k = 0
cg_traj_pos = []
for ts in universe_aa_trajs[universe_index].trajectory:
    
    
    _pos = []
    for i in range(1, 301):
    
        idx = np.where(aa_cg_map.map == i)
    #print(idx[0])
    #for j in range(3):
        
        _pos.append( ts.positions[idx].mean(axis=0) )
        
    cg_traj_pos.append(_pos)


cg_traj_pos = np.array(cg_traj_pos)
### update universe
cg_universe.load_new(cg_traj_pos, format=MemoryReader)


<Universe with 300 atoms>

In [88]:

cg_universe.trajectory.n_frames


5172

In [89]:

universe_aa_trajs[universe_index].trajectory.n_frames


5172

#### visualize CG trajectory

In [90]:
view = nv.show_mdanalysis(cg_universe)

# clear representations
#view.clear_representations()

#view.add_representation('spacefill')
view._remote_call("setSize", target="Widget", args=["1000px", "500px"])
#?view.center_view()

# specify color
view.add_representation('spacefill', selection=np.arange(169), color='blue', radius=1.5, opacity=0.85)
view.add_representation('spacefill', selection=np.arange(169,300), color='red', radius=1.5, opacity=0.85)
view.add_representation('cartoon', color='black')

# specify color
#view.add_representation('spacefill', selection=list(np.arange(169)), color='blue', radius=5, opacity=0.85)
#view.add_representation('spacefill', selection=list(np.arange(169, 300)), color='red', radius=5)

#view.add_representation('spacefill', selection="index 100 to 299", color='red')

#view.component_1.add_surface(opacity=0.3)
#view.download_image()
#view.render_image(factor=6) # higher is better
# need to run below command in different notebook cell
#view._display_image()

view

NGLWidget(max_frame=5171)

In [91]:
#### save to memory
cg_atoms = cg_universe.select_atoms("name CG")
cg_atoms.write("integrin/all-atom/cg_open.pdb")

/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:753: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:11

In [92]:
cg_atoms.write('integrin/all-atom/cg_open.xtc', frames='all')

In [93]:
##### load from memory and check
check_cg_universe = mda.Universe('integrin/all-atom/cg_open.pdb', 'integrin/all-atom/cg_open.xtc')

/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:300: RuntimeWarning: invalid value encountered in scalar divide
  alpha = np.rad2deg(np.arccos(np.dot(y, z) / (ly * lz)))
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:301: RuntimeWarning: invalid value encountered in scalar divide
  beta = np.rad2deg(np.arccos(np.dot(x, z) / (lx * lz)))
/home1/03273/tg825722/.local/lib/python3.9/site-packages/MDAnalysis/lib/mdamath.py:302: RuntimeWarning: invalid value encountered in scalar divide
  gamma = np.rad2deg(np.arccos(np.dot(x, y) / (lx * ly)))


In [94]:
check_cg_universe

<Universe with 300 atoms>

In [95]:
check_cg_universe.atoms.n_atoms

300

In [96]:
view = nv.show_mdanalysis(check_cg_universe)

# clear representations
#view.clear_representations()

#view.add_representation('spacefill')
view._remote_call("setSize", target="Widget", args=["1000px", "500px"])
#?view.center_view()

# specify color
view.add_representation('spacefill', selection=np.arange(169), color='blue', radius=1.5, opacity=0.85)
view.add_representation('spacefill', selection=np.arange(169,300), color='red', radius=1.5, opacity=0.85)
view.add_representation('cartoon', color='black')

# specify color
#view.add_representation('spacefill', selection=list(np.arange(169)), color='blue', radius=5, opacity=0.85)
#view.add_representation('spacefill', selection=list(np.arange(169, 300)), color='red', radius=5)

#view.add_representation('spacefill', selection="index 100 to 299", color='red')

#view.component_1.add_surface(opacity=0.3)
#view.download_image()
#view.render_image(factor=6) # higher is better
# need to run below command in different notebook cell
#view._display_image()

view

NGLWidget(max_frame=5171)

In [145]:
universe_aa_cg_traj.trajectory[0].positions.shape

(1749, 3)

#### read aa to cg mapping and save to .tex file

In [69]:
####
select_map = 'ba_bent'

aa_cg_map = pd.read_csv('../../../integrin/all-atom/' + select_map + '.dat', names=['Bead'])
aa_cg_map['Residue'] = aa_cg_map.index
aa_cg_map = aa_cg_map[['Residue', 'Bead']]


# Grouping by 'Col1' and combining 'Col2' values
#aa_cg_map_grouped = aa_cg_map.groupby('Bead')['Residue'].apply(lambda x: ', '.join(map(str, x))).reset_index()
aa_cg_map_grouped = aa_cg_map.groupby('Bead')['Residue'].apply(lambda x: f"{x.min()} - {x.max()}").reset_index()

def reshape(df, rows=60):
    length = len(df)
    cols = np.ceil(length / rows).astype(int)
    df = df.assign(rows=np.tile(np.arange(rows), cols)[:length], 
                   cols=np.repeat(np.arange(cols), rows)[:length]) \
           .pivot('rows', 'cols', df.columns.tolist()) \
           .sort_index(level=1, axis=1).droplevel(level=1, axis=1).rename_axis(None)
    return df

aa_cg_map_grouped_wrapped = reshape(aa_cg_map_grouped)

/tmp/ipykernel_9128/2647826261.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df = df.assign(rows=np.tile(np.arange(rows), cols)[:length],


In [65]:
#aa_cg_map_grouped_wrapped

In [66]:
for c in range(11):
    with open('si_table_1.tex','w') as tf:
        styler = aa_cg_map_grouped_wrapped.style.hide(axis='index')
        tf.write(styler.to_latex(caption=r'Mapping of residues in bent-closed, extended-Int 1, and extended-Int 2 state to 300 bead CG model.',                                 
                       label='table'+str(c), hrules=True,
                       environment="longtable",
                       column_format='|p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}|'))
        

In [ ]:
####
select_map = 'ba_open'

aa_cg_map = pd.read_csv('../../../integrin/all-atom/' + select_map + '.dat', names=['Bead'])
aa_cg_map['Residue'] = aa_cg_map.index
aa_cg_map = aa_cg_map[['Residue', 'Bead']]


# Grouping by 'Col1' and combining 'Col2' values
#aa_cg_map_grouped = aa_cg_map.groupby('Bead')['Residue'].apply(lambda x: ', '.join(map(str, x))).reset_index()
aa_cg_map_grouped = aa_cg_map.groupby('Bead')['Residue'].apply(lambda x: f"{x.min()} - {x.max()}").reset_index()

def reshape(df, rows=60):
    length = len(df)
    cols = np.ceil(length / rows).astype(int)
    df = df.assign(rows=np.tile(np.arange(rows), cols)[:length], 
                   cols=np.repeat(np.arange(cols), rows)[:length]) \
           .pivot('rows', 'cols', df.columns.tolist()) \
           .sort_index(level=1, axis=1).droplevel(level=1, axis=1).rename_axis(None)
    return df

aa_cg_map_grouped_wrapped = reshape(aa_cg_map_grouped)

In [71]:
for c in range(11):
    with open('si_table_2.tex','w') as tf:
        styler = aa_cg_map_grouped_wrapped.style.hide(axis='index')
        tf.write(styler.to_latex(caption=r'Mapping of residues in extended-open state to 300 bead CG model.',                                 
                       label='table'+str(c), hrules=True,
                       environment="longtable",
                       column_format='|p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}||p{0.6cm}|p{1.45cm}|'))
        